### Tool Calling

In [2]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()


True

### Tool Creation

In [3]:
@tool
def multiply(a: int , b: int) -> int:
    """Given 2 numbers a and b, this tool returns their product"""
    return a * b


In [5]:
print(multiply.invoke({'a': 3, 'b': 8}))

24


In [7]:
multiply.name

'multiply'

In [8]:
multiply.description

'Given 2 numbers a and b, this tool returns their product'

In [11]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}